In [1]:
import pandas as pd
import numpy as np
import re
import os
from datetime import datetime as dt
from scipy.spatial import distance
import scipy.stats as stats
from geopy.distance import distance as geodist

%matplotlib inline

In [4]:
def bt_file_process(filepath):
    
    new_files_dir = "_generated"
    
    with open(filepath, encoding='utf-8') as f:
        lines = f.readlines()
        
    if os.path.exists(os.path.join(os.getcwd(), new_files_dir)) is False:
        os.mkdir(os.path.join(os.getcwd(), new_files_dir))
    
    new_file_path = os.path.join(os.getcwd(), new_files_dir, os.path.basename(filepath))
    new_le_file_path = os.path.join(os.getcwd(), new_files_dir, "_".join(["le", os.path.basename(filepath)]))
    
    with open(new_file_path, 'w+', encoding='utf-8') as f:
        for line, i in zip(lines, range(len(lines))):
            if line.find(';LE;') == -1:
                f.write(line)
    
    with open(new_le_file_path, 'w+', encoding='utf-8') as f:
        for line, i in zip(lines, range(len(lines))):
            if line.find(';LE;') != -1:
                f.write(line)
                
    return {'BASE': new_file_path, 'LE': new_le_file_path}

In [5]:
def bt_make_dataframes(file_path, le_file_path, sampling_freq, rolling = False, sampling = True):
    df = pd.read_csv(file_path, index_col = False, header = None, low_memory = False, \
                         names = ['timestamp', 'ssid', 'bssid', 'major_class', 'class', \
                                      'bond_state', 'type'])
    
    new_files_dir = "_datasets"
    if os.path.exists(os.path.join(os.getcwd(), new_files_dir)) is False:
        os.mkdir(os.path.join(os.getcwd(), new_files_dir))
        
    new_files_dir += "\\" + sampling_freq
    if os.path.exists(os.path.join(os.getcwd(), new_files_dir)) is False:
        os.mkdir(os.path.join(os.getcwd(), new_files_dir))

    df['timestamp'] = df['timestamp'].apply(lambda x: dt.strptime(x, '%d.%m.%Y_%H:%M:%S.%f'))

    one_hot_maj_class = pd.get_dummies(df['major_class'], prefix='one_hot_c')
    df = df.join(one_hot_maj_class)

    one_hot_bond = pd.get_dummies(df['bond_state'], prefix='one_hot_b')
    df = df.join(one_hot_bond)

    one_hot_type = pd.get_dummies(df['type'], prefix='one_hot_t')
    df = df.join(one_hot_type)

    df.index = pd.DatetimeIndex(df.timestamp)
    df = df.sort_index()

    df = df.drop(['timestamp', 'ssid', 'class', 'major_class', 'bond_state', 'type'], axis = 1)

    bssid_map = { bssid.replace(' ', ''): idx for bssid, idx in zip(df.bssid.unique(), range(len(df.bssid.unique()))) }
    df.bssid = df.bssid.apply(lambda x: str(x).replace(' ', ''))
    df['count'] = 1

    def agg_string_join(col):
        col = col.apply(lambda x: str(x))
        return col.str.cat(sep = ',').replace(' ', '')

    def agg_bssid_col(col):
        array_len = len(bssid_map)
        array = np.zeros(array_len, dtype = 'int8')
        def fill_array(bssid):
            array[bssid_map[bssid.replace(' ', '')]] = 1
            return

        col.apply(lambda x: fill_array(x))
        return np.array2string(array, separator = ',').replace(' ', '')[1:-1]
    
    one_hot_columns_count = 0
    for col in df.columns:
        if col.find('one_hot') != -1:
            one_hot_columns_count += 1

    cat_columns = df.columns[1:1 + one_hot_columns_count]
    cat_columns_map = { col: 'mean' for col in cat_columns }

    all_func_dicts_quantum = { 'bssid' : agg_bssid_col, 'count' : 'sum' }
    all_func_dicts_quantum.update(cat_columns_map)

    df_quantum = df.groupby(pd.Grouper(freq = '5s'), as_index=True).agg(all_func_dicts_quantum)

    df_quantum = df_quantum.reset_index()
    df_quantum.index = pd.DatetimeIndex(df_quantum.timestamp)

    df_quantum = df_quantum.dropna()

    df_le = pd.read_csv(le_file_path, index_col = False, header = None, low_memory = False, \
                        names = ['timestamp', 'stamp', '1', '2', 'level', '3', 'connectable', '8'])

    df_le = df_le.drop(df_le.columns.difference(['connectable','timestamp', 'level']), axis = 1)
    df_le['timestamp'] = df_le['timestamp'].apply(lambda x: dt.strptime(x, '%d.%m.%Y_%H:%M:%S.%f'))
    df_le.index = pd.DatetimeIndex(df_le.timestamp)
    df_le = df_le.sort_index()

    df_le['connectable'] = df_le['connectable'].apply(lambda x: 1 if str(x).lower() == 'true' else 0)

    df_le = df_le.groupby(pd.Grouper(freq = '5s'), as_index=True).agg({'level':'mean', 'connectable':'mean'})

    df_le = df_le.dropna()

    def get_le_conn_status_from_row(row):
        conn = df_le.iloc[df_le.index.get_loc(row.name, method = 'nearest')]['connectable']
        time = df_le.iloc[df_le.index.get_loc(row.name, method = 'nearest')].name
        return conn if abs((time - row.name).total_seconds()) < 10 else 0

    def get_le_level_from_row(row):
        level = df_le.iloc[df_le.index.get_loc(row.name, method = 'nearest')]['level']
        time = df_le.iloc[df_le.index.get_loc(row.name, method = 'nearest')].name
        return level if abs((time - row.name).total_seconds()) < 10 else 0


    df_quantum['le_connectable'] = df_quantum.apply(lambda row: get_le_conn_status_from_row(row), axis = 1)
    df_quantum['le_level'] = df_quantum.apply(lambda row: get_le_level_from_row(row), axis = 1)

    def string2array(string):
        try:
            array = np.fromstring(string, sep=',')
            return array
        except:
            return np.nan

    def to_ones_array(array):
        try:
            array[array != 0] = 1
            return array
        except:
            return np.nan

    def get_len(obj):
        try:
            length = len(obj)
            return length
        except:
            return np.nan

    def get_occured_nets_count(row, prev_col, curr_col):
        prev = to_ones_array(string2array(row[prev_col]))
        curr = to_ones_array(string2array(row[curr_col]))
        intersection = np.logical_and(curr, prev)
        diff = np.logical_and(curr, np.logical_not(intersection))

        if (np.count_nonzero(np.logical_or(prev, curr)) == 0):
            return 0

        return np.count_nonzero(diff) / np.count_nonzero(np.logical_or(prev, curr))

    def get_disappeared_nets_count(row, prev_col, curr_col):
        prev = to_ones_array(string2array(row[prev_col]))
        curr = to_ones_array(string2array(row[curr_col]))
        intersection = np.logical_and(curr, prev)
        diff = np.logical_and(prev, np.logical_not(intersection))

        if (np.count_nonzero(np.logical_or(prev, curr)) == 0):
            return 0

        return np.count_nonzero(diff) / np.count_nonzero(np.logical_or(prev, curr))

    def get_jaccard_index(row, prev_col, curr_col):
        prev = to_ones_array(string2array(row[prev_col]))
        curr = to_ones_array(string2array(row[curr_col]))
        return distance.jaccard(prev, curr)

    def get_occur_speed(row, prev_col, curr_col):
        prev = to_ones_array(string2array(row[prev_col]))
        curr = to_ones_array(string2array(row[curr_col]))
        return np.linalg.norm(prev - curr) / np.sqrt(get_len(prev))

    def get_level_speed(row, prev_col, curr_col):
        prev = string2array(row[prev_col])
        curr = string2array(row[curr_col])
        return np.linalg.norm(prev - curr) / np.sqrt(get_len(prev))

    def calc_single_cols_in_window(df, col, new_col, window, func):
        def func_wrapper(func, row, prev_col, curr_col):
            delta = row.timestamp - row.prev_timestamp
            if pd.isnull(delta):
                delta = 0
            else:
                delta = abs(delta.total_seconds())
            if delta > 10 * 60:
                return np.nan
            else:
                return func(row, prev_col_name, col)

        new_cols = []

        for i in range(window):
            prev_col_name = "_".join(['prev', col, str(i + 1)])
            new_col_name = "_".join([new_col, str(i + 1)])

            df.loc[:, 'prev_timestamp'] = df.timestamp.shift(i + 1)
            df.loc[:, prev_col_name] = df[col].shift(i + 1)
            df.loc[:, new_col_name] = df.apply(lambda row: func_wrapper(func, row, prev_col_name, col), axis = 1)
            df = df.drop(prev_col_name, axis = 1)
            df = df.drop('prev_timestamp', axis = 1)
            new_cols.append(new_col_name)

        df.loc[:, "_".join([new_col, 'mean'])] = df[new_cols].mean(axis = 1)
        df.loc[:, "_".join([new_col, 'median'])] = df[new_cols].median(axis = 1)
        df.loc[:, "_".join([new_col, 'var'])] = df[new_cols].var(axis = 1)

        return df

    WINDOW_SIZE = 3

    occur_and_level_columns_map = [
        ("bssid", "occured_devices_count", WINDOW_SIZE, get_occured_nets_count),
        ("bssid", "disappeared_devices_count", WINDOW_SIZE, get_disappeared_nets_count),
        ("bssid", "jaccard_index", WINDOW_SIZE, get_jaccard_index), 
        ("bssid", "occur_speed", WINDOW_SIZE, get_occur_speed)
    ]

    for (col, new_col, window, func) in occur_and_level_columns_map:
        df_quantum = calc_single_cols_in_window(df_quantum, col, new_col, window, func)

    def get_conn_level_speed(row, prev_col, curr_col):
        return row[curr_col] - row[prev_col]

    WINDOW_SIZE = 3

    single_columns_map = [
        ("count", "count_speed", WINDOW_SIZE, get_conn_level_speed)
    ]

    for (col, new_col, window, func) in single_columns_map:
        df_quantum = calc_single_cols_in_window(df_quantum, col, new_col, window, func)

    def get_acceleration(row, prev_col, curr_col):
        return abs(row[curr_col] - row[prev_col])

    WINDOW_SIZE = 3

    multi_speed_cols = ["occured_devices_count_mean", "jaccard_index_mean", "occur_speed_mean", "disappeared_devices_count_mean", 
                        "count_speed_mean"]

    for col in multi_speed_cols:
        df_quantum = calc_single_cols_in_window(df_quantum, col, "_".join(["acceleration", col]), window, func)

    def agg_str(col):
        all_freq = col.str.cat(sep=',')
        return string2array(all_freq)

    def str_mean(col):
        return np.mean(agg_str(col))

    def str_var(col):
        return np.var(agg_str(col))

    def str_median(col):
        return np.median(agg_str(col))

    def str_skew(col):
        return stats.skew(agg_str(col))

    def str_kurt(col):
        return stats.kurtosis(agg_str(col))

    def mean(col):
        return np.mean(col)

    def var(col):
        return np.var(col)

    def median(col):
        return np.median(col)

    def skew(col):
        return stats.skew(col)

    def kurt(col):
        return stats.kurtosis(col)

    df_quantum = df_quantum.drop(['bssid', 'timestamp'], axis = 1)

    common_cols = df_quantum.columns[:one_hot_columns_count + 3]
    speed_acc_cols = df_quantum.columns[one_hot_columns_count + 3:]

    common_funcs_list = [mean, var, median, skew, kurt]
    special_funcs_list = [mean, pd.DataFrame.mad, skew]

    common_cols_map = { col : common_funcs_list for col in common_cols }
    speed_acc_cols_map = { col : special_funcs_list for col in speed_acc_cols }

    agg_dict = common_cols_map
    agg_dict.update(speed_acc_cols_map)

    df_quantum[speed_acc_cols] = df_quantum[speed_acc_cols].apply(pd.to_numeric)

    df_sampling = df_quantum.groupby(pd.Grouper(freq = sampling_freq)).agg(agg_dict)
    df_sampling.columns = ["_".join([str(high_level_name), str(low_level_name)]) \
                               for (high_level_name, low_level_name) in df_sampling.columns.values]
    
    df_sampling = df_sampling.dropna()
    df_sampling = df_sampling.fillna(0)
    
    index = os.path.basename(file_path).split('_')[-1][0]
    
    df_sampling.to_csv(new_files_dir + "\\bt_sampling_dataset_" + index + ".csv")
    
    df_rolling = df_quantum.rolling(sampling_freq, min_periods = 1, center = False).agg(agg_dict)
    
    df_rolling.columns = ["_".join([str(high_level_name), str(low_level_name)]) \
                              for (high_level_name, low_level_name) in df_rolling.columns.values]

    df_rolling = df_rolling.dropna()
    df_rolling = df_rolling.fillna(0)

    df_rolling.to_csv(new_files_dir + "\\bt_rolling_dataset_" + index + ".csv")

In [6]:
def bt_pipeline(files, sampling_freq):
    logs = []
    for file in files:
        logs.append(bt_file_process(file))
    for t in logs:
        print(t['BASE'], sampling_freq)
        bt_make_dataframes(t['BASE'], t['LE'], sampling_freq, True, True)

In [7]:
SAMPLING_FREQs = ['5s', '10s', '30s', '60s', '90s', '120s', '240s', '600s']
data_list = [
    ".\\raw_data\\bt_1.data",
    ".\\raw_data\\bt_2.data",
    ".\\raw_data\\bt_3.data",
    ".\\raw_data\\bt_4.data",
    ".\\raw_data\\bt_5.data",
    ".\\raw_data\\bt_6.data",
    ".\\raw_data\\bt_7.data",
]

In [ ]:
for freq in SAMPLING_FREQs:
    bt_pipeline(data_list, freq)

D:\Dev\nir\_generated\bt_1.data 5s


D:\Program Files\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\Program Files\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
D:\Program Files\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3405: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


D:\Dev\nir\_generated\bt_2.data 5s


D:\Program Files\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\Program Files\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
D:\Program Files\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3405: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


D:\Dev\nir\_generated\bt_3.data 5s


D:\Program Files\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\Program Files\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


D:\Dev\nir\_generated\bt_4.data 5s
D:\Dev\nir\_generated\bt_5.data 5s


D:\Program Files\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\Program Files\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
D:\Program Files\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3405: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


In [ ]:
# bt_pipeline([".\\user_1\\wifi\\wifi_0.data"], SAMPLING_FREQ)

In [ ]:
# bt_file_process(".\\bt_7.data")